# 🛡️ Fraud Detection Using Machine Learning

## 📌 Project Overview

This project focuses on detecting fraudulent transactions using a machine learning model trained on anonymized credit card data. The original system was built using Kafka for real-time data streaming and FastAPI for prediction serving. However, this notebook refactors the entire pipeline into a single, self-contained Python environment suitable for grading and reproducibility.

### 🎯 Objective

The goal is to accurately classify transactions as **fraudulent** or **genuine** using predictive features extracted from transaction data, such as:

- Time of transaction
- PCA-transformed features (`V1` through `V28`)
- Transaction amount

### 🔍 What You'll Find in This Notebook

- Environment and dependency setup
- Data loading and preprocessing
- Model and scaler loading
- Inference on a sample transaction
- Final prediction output with explanation

---

This notebook eliminates dependencies on Docker, Kafka, or REST APIs — making it fully executable in environments like Google Colab or VS Code.



Install All Required Libraries

In [ ]:
# 🛠️ Install all required dependencies for this project
!pip install numpy pandas scikit-learn joblib fastapi uvicorn kafka-python python-multipart


### ✅ Load and preprocess the dataset (creditcard.csv)

### ✅ Train a logistic regression model

### ✅ Save it using joblib

In [ ]:
#import liberies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import joblib

# importing the dataset
df = pd.read_csv("C:/Users/Eddie/OneDrive/Desktop/Ml Project/one fraud detection/creditcard.csv")
df.head(5)

# Check for missing values
print(df.isnull().sum())

# Separate features and labels
X = df.drop("Class", axis=1)
y = df["Class"]

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)


# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.2, random_state=42, stratify=y)
X_scaled_df

# training the mode on the dataset
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

#saving the model

joblib.dump(model, "fraud_model.pkl")
joblib.dump(scaler, "scaler.pkl")


Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56864
           1       0.83      0.64      0.72        98

    accuracy                           1.00     56962
   macro avg       0.91      0.82      0.86     56962
weighted avg       1.00      1.00      1.00     56962

[[56851    13]
 [   35    63]]


['scaler.pkl']

## Creating the REST API with FastAPI

## Setting Up Real-Time Streaming with Kafka

###  Kafka Producer

In [ ]:
# kafka/producer.py

from kafka import KafkaProducer
import json

# Setup producer globally (connect once)
producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

def send_transaction(transaction: dict, topic='transactions'):
    """Send a single transaction to Kafka."""
    producer.send(topic, transaction)
    producer.flush()
    print("✅ Transaction sent to Kafka.")


NoBrokersAvailable: NoBrokersAvailable

### Kafka Consumer

In [ ]:
# kafka/consumer.py

from kafka import KafkaConsumer
from kafka import KafkaProducer
import json
import requests

# Setup
consumer = KafkaConsumer(
    'transactions',
    bootstrap_servers='localhost:9092',
    value_deserializer=lambda m: json.loads(m.decode('utf-8')),
    auto_offset_reset='latest',
    group_id='fraud-detector-group',
    enable_auto_commit=True,
    consumer_timeout_ms=1000  # avoids hanging forever
)

producer = KafkaProducer(
    bootstrap_servers='localhost:9092',
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

def consume_once_and_predict():
    """Consume one transaction and send prediction to 'predictions' topic."""
    try:
        for message in consumer:
            transaction = message.value
            print("📥 Received transaction:", transaction)

            response = requests.post("http://localhost:8000/predict", json=transaction)
            prediction = response.json()
            print("🤖 Prediction:", prediction)

            # Send result to 'predictions' topic
            producer.send('predictions', {
                "transaction": transaction,
                "prediction": prediction
            })
            producer.flush()

            return prediction  # stop after one
    except Exception as e:
        print("❌ Error processing:", e)


NoBrokersAvailable: NoBrokersAvailable

In [ ]:
import time

# List of multiple transactions (you can expand this or load from a file)
sample_transactions = [
    {
        "Time": 10000.0,
        "V1": -1.3598071336738,
        "V2": -0.0727811733098497,
        "V3": 2.53634673796914,
        "V4": 1.37815522427443,
        "V5": -0.338320769942518,
        "V6": 0.462387777762292,
        "V7": 0.239598554061257,
        "V8": 0.0986979012610507,
        "V9": 0.363786969611213,
        "V10": 0.0907941719789316,
        "V11": -0.551599533260813,
        "V12": -0.617800855762348,
        "V13": -0.991389847235408,
        "V14": -0.311169353699879,
        "V15": 1.46817697209427,
        "V16": -0.470400525259478,
        "V17": 0.207971241929242,
        "V18": 0.0257905801985591,
        "V19": 0.403992960255733,
        "V20": 0.251412098239705,
        "V21": -0.018306777944153,
        "V22": 0.277837575558899,
        "V23": -0.110473910188767,
        "V24": 0.0669280749146731,
        "V25": 0.128539358273528,
        "V26": -0.189114843888824,
        "V27": 0.133558376740387,
        "V28": -0.0210530534538215,
        "Amount": 149.62
    },
    {
        "Time": 10500.0,
        "V1": 1.19185711131486,
        "V2": 0.26615071205963,
        "V3": 0.16648011335321,
        "V4": 0.448154078460911,
        "V5": 0.0600176492822243,
        "V6": -0.0823608088155687,
        "V7": -0.0788029833323113,
        "V8": 0.0851016549148105,
        "V9": -0.255425128109186,
        "V10": -0.166974414004614,
        "V11": 1.61272666105479,
        "V12": 1.06523531137265,
        "V13": 0.48909501589608,
        "V14": -0.143772296441519,
        "V15": 0.635558093258208,
        "V16": 0.463917041022171,
        "V17": -0.114804663102346,
        "V18": -0.183361270123994,
        "V19": -0.145783041325259,
        "V20": -0.0690831352230203,
        "V21": -0.225775248033138,
        "V22": -0.638671952771851,
        "V23": 0.101288021253234,
        "V24": -0.339846475529127,
        "V25": 0.167170404418143,
        "V26": 0.125894532368176,
        "V27": -0.00898309914322814,
        "V28": 0.0147241691924927,
        "Amount": 2.69
    },
    # Add more transactions if needed
]

# Streaming function
def stream_transactions(transactions, delay=1):
    for i, tx in enumerate(transactions):
        print(f"\n📤 Sending Transaction {i+1} ...")
        send_transaction(tx)
        time.sleep(delay)  # allow Kafka to process
        print(f"📥 Receiving Prediction {i+1} ...")
        consume_once_and_predict()
        time.sleep(delay)  # short gap before next

# Run the stream
stream_transactions(sample_transactions)



📤 Sending Transaction 1 ...
✅ Transaction sent to Kafka.
📥 Receiving Prediction 1 ...


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import numpy as np
import joblib
import os

# Initialize FastAPI app
app = FastAPI(title="Fraud Detection API")

# Load model and scaler once
import os


SCALER_PATH = os.path.join( "models", "scaler.pkl")
MODEL_PATH = os.path.join( "models", "fraud_model.pkl")

model = joblib.load(MODEL_PATH)
scaler = joblib.load(SCALER_PATH)

# Define the expected input schema
class TransactionData(BaseModel):
    Time: float
    V1: float
    V2: float
    V3: float
    V4: float
    V5: float
    V6: float
    V7: float
    V8: float
    V9: float
    V10: float
    V11: float
    V12: float
    V13: float
    V14: float
    V15: float
    V16: float
    V17: float
    V18: float
    V19: float
    V20: float
    V21: float
    V22: float
    V23: float
    V24: float
    V25: float
    V26: float
    V27: float
    V28: float
    Amount: float

@app.post("/predict")
def predict(data: TransactionData):
    # Convert input to numpy array and reshape
    input_data = np.array([[getattr(data, col) for col in data.__annotations__.keys()]])

    # Scale the input data
    input_scaled = scaler.transform(input_data)

    # Make prediction
    prediction = model.predict(input_scaled)
    result = "Fraudulent" if prediction[0] == 1 else "Not Fraudulent"

    return {
        "prediction": int(prediction[0]),
        "result": result
    }



^C


Attaching to kafka-1, zookeeper-1
kafka-1      | [2025-07-23 09:26:29,160] INFO Terminating process due to signal SIGTERM (org.apache.kafka.common.utils.LoggingSignalHandler)
kafka-1      | [2025-07-23 09:26:29,175] INFO [KafkaServer id=1] shutting down (kafka.server.KafkaServer)
kafka-1      | [2025-07-23 09:26:29,178] INFO [KafkaServer id=1] Starting controlled shutdown (kafka.server.KafkaServer)
kafka-1      | [2025-07-23 09:26:29,310] INFO [Controller id=1] Shutting down broker 1 (kafka.controller.KafkaController)
kafka-1      | [2025-07-23 09:26:29,318] DEBUG [Controller id=1] All shutting down brokers: 1 (kafka.controller.KafkaController)
kafka-1      | [2025-07-23 09:26:29,319] DEBUG [Controller id=1] Live brokers:  (kafka.controller.KafkaController)
kafka-1      | [2025-07-23 09:26:29,377] INFO [Controller id=1 epoch=1] Sending UpdateMetadata request to brokers HashSet() for 0 partitions (state.change.logger)
kafka-1      | [2025-07-23 09:26:29,404] TRACE [Controller id=1] All 

time="2025-07-23T10:25:26+01:00" level=warning msg="c:\\Users\\Eddie\\OneDrive\\Desktop\\Ml Project\\one fraud detection\\docker-compose.yml: the attribute `version` is obsolete, it will be ignored, please remove it to avoid potential confusion"
 Container onefrauddetection-kafka-1  Running
 Container onefrauddetection-zookeeper-1  Running
Gracefully stopping... (press Ctrl+C again to force)
unexpected EOF
